Import and setup some auxiliary functions

In [1]:
import torch
from torchvision import datasets, transforms
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat


def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, x_train, y_train, x_test, y_test, n_classes, device):
    print('Running...')        
    if device != 'cpu' and torch.cuda.is_available():
        device = torch.device("cuda")
        print('Training on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Training on CPU')
        
    start = timeit.default_timer()
    np.random.seed(0)
    predicted_y_test = algorithm(x_train, y_train, x_test, n_classes, device)
    np.random.seed()
    stop = timeit.default_timer()
    run_time = stop - start
    correct_predict = (y_test
                       == predicted_y_test).astype(np.int32).sum()
    incorrect_predict = len(y_test) - correct_predict
    accuracy = float(correct_predict) / len(y_test)

    print('Correct Predict: {}/{} total \tAccuracy: {:5f} \tTime: {:2f}'.format(correct_predict,
                                                                                len(y_test), accuracy, run_time))
    return correct_predict, accuracy, run_time

TODO: Implement knn here

In [2]:
def knn(x_train, y_train, x_test, n_classes, device):
    """
    x_train: 60000 x 784 matrix: each row is a flattened image of an MNIST digit
    y_train: 60000 vector: label for x_train
    x_test: 1000 x 784 testing images
    n_classes: no. of classes in the classification task
    device: pytorch device on which to run the code
    return: predicted y_test which is a 1000-sized vector
    """
    #raise NotImplementedError
    x_train = torch.from_numpy(x_train).float().to(device)
    y_train = torch.from_numpy(y_train).float().to(device)
    x_test = torch.from_numpy(x_test).float().to(device)

    k_value = 5

    euclidean_dist = torch.cdist(x_train,x_test)  #60,000 * 1,000
    euclidean_dist = torch.transpose(euclidean_dist,0,1)  #1,000 * 60,000
    
    result = torch.ones((1000,1),device=device)
    count = 0
    for row in euclidean_dist: #1000
      a,inde = torch.topk(row,k_value,largest=False)
      #inde = b.data # indeces of k training

      temp = torch.ones((k_value,1),device = device)
      a = 0
      for i in inde:
        temp[a] = y_train[i]
        a+=1
      result[count] = torch.mode(temp,0)[0]
      count += 1
    
    result = result.cpu().numpy()
    return np.squeeze(result)
  


Main loop. You can only run this after filling the knn function above

In [3]:
min_thres = 0.84
max_thres = 0.94
n_classes = 10
# change to cpu to run on CPU
device = 'gpu'

mnist_train = datasets.MNIST('data', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.Normalize((0.1307,), (0.3081,)),
                             ])
                             )
mnist_test = datasets.MNIST('data', train=False, download=True,
                             transform=transforms.Compose([
                                 transforms.Normalize((0.1307,), (0.3081,)),
                             ])
                            )
result = [OrderedDict(first_name='Insert your First name here',
                      last_name='Insert your Last name here')]

# convert pytorch tensors to numpy arrays
(x_train, y_train) = (mnist_train.data.cpu().numpy(), mnist_train.targets.cpu().numpy())
(x_valid, y_valid) = (mnist_test.data.cpu().numpy(), mnist_test.targets.cpu().numpy())

# flatten 28x28 images into 784 sized vectors
x_train = x_train.reshape(x_train.shape[0], -1)
x_valid = x_valid.reshape(x_valid.shape[0], -1)

# You may want to use a smaller training set to save time when debugging
# i.e.: Put something like:
# (x_train, y_train) = (x_train[:5000], y_train[:5000])

# For this assignment, we only test on the first 1000 samples of the test set
(x_valid, y_valid) = (x_valid[:1000], y_valid[:1000])

print("Dimension of dataset: ")
print("Train:", x_train.shape, y_train.shape, "\nTest:", x_valid.shape, y_valid.shape)

(correct_predict, accuracy, run_time) = run(knn, x_train, y_train, x_valid, y_valid, n_classes, device)
score = compute_score(accuracy, min_thres, max_thres)
result = OrderedDict(correct_predict=correct_predict,
                     accuracy=accuracy, score=score,
                     run_time=run_time)

with open('result.txt', 'w') as f:
    f.writelines(pformat(result, indent=4))

print(pformat(result, indent=4))

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Dimension of dataset: 
Train: (60000, 784) (60000,) 
Test: (1000, 784) (1000,)
Running...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Training on GPU: Tesla K80
Correct Predict: 961/1000 total 	Accuracy: 0.961000 	Time: 9.540223
OrderedDict([   ('correct_predict', 961),
                ('accuracy', 0.961),
                ('score', 100.0),
                ('run_time', 9.540222857999993)])
